# DST Clustering
An attempt to use HAPI-NN to determine cluster different DST events.

In [1]:
try:
    import hapi_nn
except:
    %pip install hapi_nn
    import os
    os._exit(00)


## Setup
Import HAPI and other packages

In [2]:
from hapiclient import hapi
from datetime import datetime
from hapi_nn import HAPINNTrainer, HAPINNTester, config
import hapi_nn
import numpy as np
import warnings
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
%matplotlib inline

2025-02-04 15:18:58.455944: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-04 15:18:58.500915: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-04 15:18:58.500934: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-04 15:18:58.500939: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-04 15:18:58.507431: I tensorflow/core/platform/cpu_feature_g

In [3]:
warnings.simplefilter('always')

### HAPI Setup
Set HAPI related parameters

In [4]:
config.MODEL_ENGINE = 'TENSORFLOW'

server = 'https://cdaweb.gsfc.nasa.gov/hapi'
dataset = 'OMNI2_H0_MRG1HR'
#Provisional Dst (2015/001-2020/366)
start = '2016-01-01T00:00:00Z'
stop = '2020-01-01T00:00:00Z'

start2 = '2015-01-01T00:00:00Z'
stop2 = '2016-01-01T00:00:00Z'

parameters = 'DST1800'
options = {'logging': True, 'usecache': True, 'cachedir': './hapicache'}

## Create HAPI-NN Trainer and Tester
Begin testing HAPI-NN

### Parameters

In [5]:
# Note the period size has effect on the duration of events that can be clustered
in_steps = 24 * 4  # 4 days
out_steps = in_steps

### Create Trainer and Tester as well as load data with HAPI

Create Trainer

In [6]:
splits = (.7, .2, .1)  # Train, Validation, Test
    
trainer = HAPINNTrainer(
    splits, in_steps, out_steps,
    preprocess_func=None,
    lag=False
)

Load data for Training

Model input will come from scalar and vector in HAPI dataset. The output comes from the first element in the column.
The input is lagged behind the output, so we are forecasting the outputs based on the inputs.

In [7]:
data, meta = hapi(server, dataset, parameters, start, stop, **options)
trainer.set_hapidatas([data], xyparameters=[['DST1800'], ['DST1800']])

hapi(): Running hapi.py version 0.2.5
hapi(): file directory = ./hapicache/cdaweb.gsfc.nasa.gov_hapi
hapi(): Reading ./hapicache/cdaweb.gsfc.nasa.gov_hapi
hapi(): Writing OMNI2_H0_MRG1HR___.json 
hapi(): Writing OMNI2_H0_MRG1HR___.pkl 
hapi(): Reading https://cdaweb.gsfc.nasa.gov/hapi/capabilities
hapi(): Writing https://cdaweb.gsfc.nasa.gov/hapi/data?id=OMNI2_H0_MRG1HR&parameters=DST1800&time.min=2016-01-01T00:00:00Z&time.max=2020-01-01T00:00:00Z&format=binary to OMNI2_H0_MRG1HR_DST1800_20160101T000000_20200101T000000.bin
hapi(): Reading and parsing OMNI2_H0_MRG1HR_DST1800_20160101T000000_20200101T000000.bin
hapi(): Writing ./hapicache/cdaweb.gsfc.nasa.gov_hapi/OMNI2_H0_MRG1HR_DST1800_20160101T000000_20200101T000000.pkl
hapi(): Writing ./hapicache/cdaweb.gsfc.nasa.gov_hapi/OMNI2_H0_MRG1HR_DST1800_20160101T000000_20200101T000000.npy


/srv/conda/envs/notebook/lib/python3.11/site-packages/hapiclient/hapitime.py:287: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  Time = pandas.to_datetime(Time, infer_datetime_format=True).tz_convert(tzinfo).to_pydatetime()


3600.0

In [8]:
len(data)

35064

Prepare the downloaded data for training

In [ ]:
trainer.prepare_data()

Test saving and loading

In [ ]:
trainer.save_prepared_data('data')

In [ ]:
trainer.load_prepared_data('./data')

Create Tester

In [ ]:
def custom_handle_2_outputs(x):
    # Tester is used for full and encoder model
    # Tester cannot handle seperate output heads, handling here
    if isinstance(x, np.ndarray):
        # handle numpy case or normal model case
        return x
    else:
        # handle encoder case with two outputs, take first
        return x[0]

tester = HAPINNTester(
    in_steps, out_steps, preprocess_func=None,
    preprocess_y_func=custom_handle_2_outputs
)

Load data for testing

In [ ]:
data, meta = hapi(server, dataset, parameters, start2, stop2, **options)
tester.set_hapidatas([data], xyparameters=[['DST1800'], ['DST1800']])

In [ ]:
len(data)

Prepare data for testing

In [ ]:
tester.prepare_data()

### Create Model

Create TensorFlow Model

In [ ]:
# Higher compression ratio the more difficult reconstructing the input will be
compression_ratio = 6
compression = in_steps // compression_ratio

In [ ]:
class VAEModel(keras.Model):
    def __init__(self, encoder, decoder, rloss_coef=1000, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.rloss_coef = rloss_coef
        self.decoder.compiled_loss.build(
            tf.zeros(self.decoder.output_shape[1:])
        )
        self.rloss = self.decoder.compiled_loss._losses[0]
        
    def train_step(self, x):
        if isinstance(x, (tuple, list)):
            x = x[0]

        with tf.GradientTape() as tape:
            z_mean, z_log_var = self.encoder(x, training=True)
            eps = tf.random.normal(shape=tf.shape(z_mean))
            z = eps * tf.exp(z_log_var * .5) + z_mean
            reconstruction_loss = self.rloss.fn(
                x, self.decoder(z, training=True)
            )
            reconstruction_loss = tf.reduce_mean(
                reconstruction_loss
            )
            divergence_loss = .5 * tf.reduce_mean(
                tf.exp(z_log_var) + z_mean**2 - 1. - z_log_var
            )
            loss = reconstruction_loss * self.rloss_coef + divergence_loss
        grads = tape.gradient(loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        return {
            'loss': loss,
            'reconstruction_loss': reconstruction_loss,
            'divergence_loss': divergence_loss
        }
    
    def call(self, inputs, training=False):
        z_mean, z_log_var = self.encoder(inputs, training=training)
        eps = tf.random.normal(shape=tf.shape(z_mean))
        z = eps * tf.exp(z_log_var * .5) + z_mean
        y = self.decoder(z, training=training)
        return y

In [ ]:
x0 = keras.layers.Input(shape=(in_steps, 1))
x = keras.layers.Conv1D(compression, 5, strides=2, padding='same', activation='swish')(x0)
x = keras.layers.Conv1D(compression * 2, 5, strides=2, padding='same', activation='swish')(x)
x = keras.layers.Conv1D(compression * 4, 5, strides=2, padding='same', activation='swish')(x)
x = keras.layers.Conv1D(compression * 8, 5, strides=2, padding='same', activation='swish')(x)
x = keras.layers.Conv1D(compression * 8, 3, strides=2, padding='same', activation='swish')(x)
x = keras.layers.Conv1D(compression * 16, 3, strides=3)(x)

x = keras.layers.Flatten()(x)
z_mean = keras.layers.Dense(compression)(x)
z_log_var = keras.layers.Dense(compression)(x)
z_log_var = keras.layers.Lambda(
    lambda x: tf.clip_by_value(x, -50, 50)  # helps to avoid NaN
)(z_log_var)
encoder_model = keras.Model(inputs=x0, outputs=[z_mean, z_log_var])
encoder_model.compile(
    optimizer='adam', loss='mse', metrics=['mae']
)
encoder_model.summary()

x0 = keras.layers.Input(shape=(compression,))
x = keras.layers.Dense(compression * 8)(x0)
x = keras.layers.Reshape((1, compression * 8))(x)
x = keras.layers.Conv1DTranspose(compression * 16, 3, strides=3)(x)
x = keras.layers.Conv1DTranspose(compression * 8, 3, strides=2, padding='same')(x)
x = keras.layers.Conv1DTranspose(compression * 8, 5, strides=2, padding='same')(x)
x = keras.layers.Conv1DTranspose(compression * 4, 5, strides=2, padding='same')(x)
x = keras.layers.Conv1DTranspose(compression * 2, 5, strides=2, padding='same')(x)
x = keras.layers.Conv1DTranspose(compression, 5, strides=2, padding='same')(x)
x = keras.layers.Conv1DTranspose(1, 1, strides=1, padding='same')(x)
decoder_model = keras.Model(inputs=x0, outputs=x)
decoder_model.compile(optimizer=keras.optimizers.Adam(.001), loss='mse', metrics=['mae'])
decoder_model.summary()

model = VAEModel(encoder_model, decoder_model, rloss_coef=2)
model.compile(optimizer=decoder_model.optimizer,
              loss=decoder_model.loss,
              metrics=decoder_model.compiled_metrics._metrics)

### Train the model

In [ ]:
epochs = 20
batch_size = 32

trainer.train(model, epochs, batch_size=batch_size)

## Utilize Model

Predict over the downloaded testing data using the default stride (out_steps)

In [ ]:
predictions = tester.test(model)
len(predictions)

Display one window of the test data overlayed with the prediction values

In [ ]:
tester.plot(predictions, 9, 'DST1800')

Display all of the test data overlayed with the prediction values

In [ ]:
tester.plot(predictions, -1, 'DST1800')

Get encodings over the downloaded testing data using the default stride (out_steps)

In [ ]:
encodings = tester.test(encoder_model)
encodings = np.array(encodings)
# note that these encoding values cannot be ploted using the tester

Plot the encodings with TSNE and PCA since the encoding vectors have more than 2 dimensions

In [ ]:
x_data = encodings.reshape(
    (encodings.shape[0], np.prod(encodings.shape[1:]))
)

In [ ]:
model = TSNE(n_components=2, init='pca', learning_rate='auto')
tsne_transformed = model.fit_transform(x_data)

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111)
ax.scatter(tsne_transformed[:, 0], tsne_transformed[:, 1])
plt.show()

In [ ]:
model = PCA(n_components=2)
pca_transformed = model.fit_transform(x_data)

fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111)
ax.scatter(pca_transformed[:, 0], pca_transformed[:, 1])
plt.show()

Cluster the test data using KMeans and GaussianMixture

KMeans has it's own transform, so plot clusters on that transform so long as only 2 clusters

In [ ]:
ndxs = np.arange(len(encodings))
pred_ndxs = np.floor(ndxs / tester.out_steps).tolist()
time_ndxs = ndxs * 60 * 60 * tester.out_steps
# assuming start date of test data is Jan. 1st, 2015
dates = [datetime.utcfromtimestamp(x) for x in (time_ndxs + datetime(year=2015, month=1, day=1).timestamp())]

In [ ]:
model = KMeans(n_clusters=2)
model.fit(x_data)
transformed = model.fit_transform(x_data)
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111)
ax.scatter(transformed[:, 0], transformed[:, 1], c=model.labels_)
for ndx in range(len(transformed)):
    if model.labels_[ndx] == 1:
        ax.annotate(dates[ndx], (transformed[ndx, 0], transformed[ndx, 1]))
plt.show()
model.labels_

Compute GaussianMixture and plot on PCA transform

In [ ]:
model = GaussianMixture(n_components=2)
model.fit(x_data)
labels = model.fit_predict(x_data)
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(111)
ax.scatter(transformed[:, 0], transformed[:, 1], c=labels)
for ndx in range(len(transformed)):
    if labels[ndx] == 1:
        ax.annotate(dates[ndx], (transformed[ndx, 0], transformed[ndx, 1]))
plt.show()
labels

Focus on a few of the two clusters by plotting them individually

In [ ]:
plt.figure(figsize=(24, 16))
for ndx, label in enumerate(labels[:len(labels)//6]):
    if label == 0:
        tester.plot(predictions, ndx, 'DST1800')
        plt.scatter(time_ndxs[ndx], 0, c='k')

In [ ]:
plt.figure(figsize=(24, 16))
for ndx, label in enumerate(labels[:len(labels)//2]):
    if label == 1:
        tester.plot(predictions, ndx, 'DST1800')
        plt.scatter(time_ndxs[ndx], 0, c='k')

### Conclusion

For the most part there seems to be a pattern in the second cluster that differentiates the two clusters. This pattern is the presences of extreme dips near the end of the series. However, the first cluster does have a couple instances with a similar pattern but these have spikes upward before the dip in their series reconstruction. The differences in the clusters does not seem to perfectly split on CMEs or Flares, with both clusters having such events.

©️ 2022 The Johns Hopkins University Applied Physics Laboratory LLC.